## Convert CSV to mseed

In [2]:
import os
import pandas as pd
import obspy as obs
import numpy as np

import matplotlib.pyplot as plt

from datetime import datetime

from andbro__write_stream_to_sds import __write_stream_to_sds

In [3]:
if os.uname().nodename == 'lighthouse':
    root_path = '/home/andbro/'
    data_path = '/home/andbro/kilauea-data/'
    archive_path = '/home/andbro/freenas/'
    bay_path = '/home/andbro/bay200/'
elif os.uname().nodename == 'kilauea':
    root_path = '/home/brotzer/'
    data_path = '/import/kilauea-data/'
    archive_path = '/import/freenas-ffb-01-data/'
    bay_path = '/bay200/'
elif os.uname().nodename == 'lin-ffb-01':
    root_path = '/home/brotzer/'
    data_path = '/import/kilauea-data/'
    archive_path = '/import/freenas-ffb-01-data/'
    bay_path = '/bay200/'

In [4]:
path_to_data = data_path+"LNM2/data/wettzell/"

### Load Data

In [8]:
files = os.listdir(path_to_data+"1sec/")
files = np.sort(files)

for file in files:
    df = pd.read_csv(path_to_data+"1sec/"+file, names=["mjd", "pressure_inside", "pressure_outside", "X"], delim_whitespace=True)

    year = file[12:16]
    doy  = file[17:20]

    date = datetime.strptime(f'{year} {doy}', '%Y %j')

    tr = obs.Trace()

    Tdelta = 1

    dat = df.pressure_outside.to_numpy()
    print(f" -> NaN: {len(dat[np.isnan(dat)])}")

    tr.data = df.pressure_outside.to_numpy()
    tr.stats.delta = Tdelta
    tr.stats.starttime = obs.UTCDateTime(str(date))
    tr.stats.network = "BW"
    tr.stats.station = "WETR"
    tr.stats.location = "01"
    tr.stats.channel = "BDO"

    st = obs.Stream(traces=tr)

    print(f" -> writing: {str(date)}")
    __write_stream_to_sds(st, path_to_data)


2054
 -> writing: 2023-07-19 00:00:00
 -> stored stream as: 2023/BW/WETR/BDO.D/BW.WETR.01.BDO.D.2023.200
2090
 -> writing: 2023-07-20 00:00:00
 -> stored stream as: 2023/BW/WETR/BDO.D/BW.WETR.01.BDO.D.2023.201
2115
 -> writing: 2023-07-21 00:00:00
 -> stored stream as: 2023/BW/WETR/BDO.D/BW.WETR.01.BDO.D.2023.202
2073
 -> writing: 2023-07-22 00:00:00
 -> stored stream as: 2023/BW/WETR/BDO.D/BW.WETR.01.BDO.D.2023.203
2151
 -> writing: 2023-07-23 00:00:00
 -> stored stream as: 2023/BW/WETR/BDO.D/BW.WETR.01.BDO.D.2023.204
2075
 -> writing: 2023-07-24 00:00:00
 -> stored stream as: 2023/BW/WETR/BDO.D/BW.WETR.01.BDO.D.2023.205
2142
 -> writing: 2023-07-25 00:00:00
 -> stored stream as: 2023/BW/WETR/BDO.D/BW.WETR.01.BDO.D.2023.206
2081
 -> writing: 2023-07-26 00:00:00
 -> stored stream as: 2023/BW/WETR/BDO.D/BW.WETR.01.BDO.D.2023.207
2104
 -> writing: 2023-07-27 00:00:00
 -> stored stream as: 2023/BW/WETR/BDO.D/BW.WETR.01.BDO.D.2023.208
2069
 -> writing: 2023-07-28 00:00:00
 -> stored stream

## Load data file

In [35]:
df = pd.read_csv(path_to_data+"luftdruck_wettzell.csv")

### Merge with Dummy Frame (to ensure constant sampling)

In [6]:
df_dummy = pd.DataFrame()

seconds = int(abs(obs.UTCDateTime(df.epoch.iloc[0][:-3]) - obs.UTCDateTime(df.epoch.iloc[-1][:-3])))

tt = []
for _n, _t in enumerate(np.arange(0, seconds+60, 60)):
    obst = obs.UTCDateTime(df.epoch.iloc[0][:-3]) + _t
    tt.append(f"{obst.date} {str(obst.time).split('.')[0]}+00")

df_dummy['epoch'] = tt

df_new = df_dummy.merge(df, how="outer", on="epoch")

In [7]:
# df_new[df_new.p.isna()]

### Create Stream and write to SDS

In [8]:
tr = obs.Trace()

Tdelta = 60

tr.data = df_new.p.to_numpy()
tr.stats.delta = Tdelta
tr.stats.starttime = obs.UTCDateTime(df.epoch.iloc[0][:-3])
tr.stats.network = "BW"
tr.stats.station = "WETR"
tr.stats.location = ""
tr.stats.channel = "BDO"

st = obs.Stream(traces=tr)

In [10]:
for datetime in pd.date_range(str(st[0].stats.starttime.date), str(st[0].stats.endtime.date)):
    st_out = st.copy()
    st_out.trim(obs.UTCDateTime(datetime), obs.UTCDateTime(datetime)+86400-Tdelta)
    # print(st_out)
    __write_stream_to_sds(st_out, path_to_data)

1 Trace(s) in Stream:
BW.WETR..BDO | 2023-09-01T00:00:00.000000Z - 2023-09-01T23:59:00.000000Z | 60.0 s, 1440 samples
 -> stored stream as: 2023/BW/WETR/BDO.D/BW.WETR..BDO.D.2023.244
1 Trace(s) in Stream:
BW.WETR..BDO | 2023-09-02T00:00:00.000000Z - 2023-09-02T23:59:00.000000Z | 60.0 s, 1440 samples
 -> stored stream as: 2023/BW/WETR/BDO.D/BW.WETR..BDO.D.2023.245
1 Trace(s) in Stream:
BW.WETR..BDO | 2023-09-03T00:00:00.000000Z - 2023-09-03T23:59:00.000000Z | 60.0 s, 1440 samples
 -> stored stream as: 2023/BW/WETR/BDO.D/BW.WETR..BDO.D.2023.246
1 Trace(s) in Stream:
BW.WETR..BDO | 2023-09-04T00:00:00.000000Z - 2023-09-04T23:59:00.000000Z | 60.0 s, 1440 samples
 -> stored stream as: 2023/BW/WETR/BDO.D/BW.WETR..BDO.D.2023.247
1 Trace(s) in Stream:
BW.WETR..BDO | 2023-09-05T00:00:00.000000Z - 2023-09-05T23:59:00.000000Z | 60.0 s, 1440 samples
 -> stored stream as: 2023/BW/WETR/BDO.D/BW.WETR..BDO.D.2023.248
1 Trace(s) in Stream:
BW.WETR..BDO | 2023-09-06T00:00:00.000000Z - 2023-09-06T23:59:0